### *Импорты* ###

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

from keras.layers import Dropout
from keras.models import load_model

### *Загрузка данных* ###

In [2]:
data = pd.read_csv('../ann/../../db/basel(1940-2023).csv')

### *Отделение признаков и таргета в отдельные датафреймы* ###

In [3]:
# Необходимо удалить предсказываемые значения из датафрейма
features = data.drop(['Temperature [2 m elevation corrected]'], axis=1)

In [4]:
target = data['Temperature [2 m elevation corrected]']

### *Разделение на обучающий и тестовый наборы* ###

Данные не размешаны и не стратицированы для того, чтобы сохранить их структуру. Их размешивание и стратификация приведут к тому,
что модель плохо распознает ежемесячные и ежегожные погодные паттерны

In [5]:
# Обучающая часть
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, stratify=None, shuffle=False)

### *Стандартизация данных* ###

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### *Инициализация модели* ###

In [7]:
model = keras.Sequential([
            layers.Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(32, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(1),
        ])

### *Компиляция модели* ###

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error')

### *Обучение модели* ###

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 10**(epoch / 20))

In [10]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])

Epoch 1/10
33097/33097 [==============================] - 97s 3ms/step - loss: 9.5181 - val_loss: 1.3193 - lr: 0.0010
Epoch 2/10
33097/33097 [==============================] - 95s 3ms/step - loss: 7.3792 - val_loss: 1.2355 - lr: 0.0011
Epoch 3/10
33097/33097 [==============================] - 95s 3ms/step - loss: 7.0303 - val_loss: 0.8806 - lr: 0.0013
Epoch 4/10
33097/33097 [==============================] - 94s 3ms/step - loss: 6.7990 - val_loss: 0.9854 - lr: 0.0014
Epoch 5/10
33097/33097 [==============================] - 96s 3ms/step - loss: 6.6390 - val_loss: 0.9516 - lr: 0.0016
Epoch 6/10
33097/33097 [==============================] - 96s 3ms/step - loss: 6.5524 - val_loss: 0.9556 - lr: 0.0018
Epoch 7/10
33097/33097 [==============================] - 95s 3ms/step - loss: 6.4657 - val_loss: 1.6805 - lr: 0.0020
Epoch 8/10
33097/33097 [==============================] - 97s 3ms/step - loss: 6.5162 - val_loss: 0.8749 - lr: 0.0022
Epoch 9/10
33097/33097 [==============================] 

### *Результативность модели* ###

In [11]:
prediction = model.predict(X_test_scaled)

print('Mean Squared Error: {:.4f}'.format(mean_squared_error(y_test, prediction)))
print('Mean Absolute Error: {:.4f}'.format(mean_absolute_error(y_test, prediction)))
print('R-squared: {:.4f}'.format(r2_score(y_test, prediction)))

2299/2299 [==============================] - 3s 1ms/step
Mean Squared Error: 1.5235
Mean Absolute Error: 1.0612
R-squared: 0.9776


### *Сохранение модели* ###

In [ ]:
# model.save('best_model.keras')

### *Загрузка модели* ###

In [ ]:
# loaded_model = load_model('best_model.keras')